In [1]:
import os
import lgsvl

INTERVAL = 0.1

sim = lgsvl.Simulator(os.environ.get("SIMULATOR_HOST", "127.0.0.1"), 8181)

print("Version =", sim.version)
print("Current Time =", sim.current_time)
print("Current Frame =", sim.current_frame)

print("Current Scene = {}".format(sim.current_scene))

# Loads the named map in the connected simulator. The available maps can be set up in web interface
if sim.current_scene == "BorregasAve":
  sim.reset()
else:
  sim.load("BorregasAve")

print("Current Scene = {}".format(sim.current_scene))

# This will print out the position and rotation vectors for each of the spawn points in the loaded map
spawns = sim.get_spawn()
for spawn in sim.get_spawn():
  print(spawn)

# Get a state from the simulator and spawn a vehicle
spawns = sim.get_spawn()

state = lgsvl.AgentState()
state.transform = spawns[0]

sim.load('BorregasAve')

forward = lgsvl.utils.transform_to_forward(spawns[0])

# Agents can be spawned with a velocity. Default is to spawn with 0 velocity
state.velocity = 20 * forward
a = sim.add_agent("Lincoln2017MKZ (Apollo 5.0)", lgsvl.AgentType.EGO, state)

Version = 2020.06
Current Time = 22.399999499321
Current Frame = 2242
Current Scene = None
Current Scene = BorregasAve
Spawn(position=Vector(-50.3400077819824, -1.03600001335144, -9.03001022338867), rotation=Vector(0, 104.823379516602, 0), destinations=[])
Spawn(position=Vector(386.800048828125, -7.73000001907349, 106.700073242188), rotation=Vector(0, 191.806182861328, 0), destinations=[])


In [2]:
BSM_PART_1_FIELDS = [
    'msgCnt',
    'id',
    'secMark',
    'latPos',  # Done
    'longPos', # Done
    'elev',    # Done
    'accuracy', # ?
    'semiMajor', # ?
    'semiMinor', # ?
    'orientation', # ?
    'tranmission', # TODO
    'speed', # Done
    'heading', # Done
    'angle', # Can't find where to access this? Can send new VehicleControl via API but cannot access?
    'longAccel', # TODO
    'latAccel', # TODO
    'vertAccel', # TODO
    'yawAccel', # TODO
    'wheelBrakes', # ?
    'traction', # ?
    'abs', # Don't think that is available
    'scs', # ^
    'brakeBoost', # Some threshold probably needed, or keep track of 'braking' history. Also, same problem as angle?
    'auxBrake', # AuxBrake, question on  access due to VehicleControl
    'width', # TODO, look at bounding box
    'length' # TODO, look at bounding box
]

class BasicSafetyMessage:
    __slots__ = BSM_PART_1_FIELDS
    def __init__(self, **kwargs):
        # Iterate through each field and set to None
        for field in self.__slots__:
            setattr(self, field, kwargs.setdefault(field, None))

    @classmethod
    def to_json(bsm):
        return json.dumps(bsm.__dict__)

In [ ]:
def BSM_part_1_gen(agent):
    state_prev = agent.state
    print(f'{state_prev}')
    state_curr = state_prev
    print(f'{state_curr}')
    while True:
        bsm = BasicSafetyMessage()
        bsm.speed = state_curr.speed
        gps_data = sim.map_to_gps(curr_state.transform)
        bsm.latPos = gps_data.latitude
        bsm.longPos = gps_data.longitude
        bsm.elevation = gps_data.altitude
        bsm.heading = gps_data.orientation
        yield bsm
        state_prev = state_curr
        state_curr = agent.state

In [ ]:
bsm_gen = BSM_part_1_gen(a)

In [ ]:
for idx in range(10):
    sim.run(INTERVAL)
    bsm = next(bsm_gen)
    print(f'{idx:03d}: {bsm.speed}')

In [ ]:
a.state.velocity = a.state.velocity - 60 * forward 

In [ ]:
a.state

In [3]:
canbus = a.get_sensors()[0]

In [4]:
canbus.enabled

True

In [5]:
canbus.transform

Transform(position=Vector(0, 0, 0), rotation=Vector(0, 0, 0))

In [ ]:
sim.map_to_gps(a.state.transform)